Chapter: Agricultural Drought Assessment in Rondônia, Brazil
Python + Google Earth Engine (GEE) script

Goal: provide a simple, functional workflow to compute a composite drought index
for a small AOI in Rondônia using CHIRPS precipitation, ERA5 temperature,
and MODIS NDVI. Outputs: drought severity map (mild/moderate/severe) and
timeseries summary for an AOI. This file is written to be runnable with the
Python Earth Engine API (ee). Adjust the date_range and AOI as needed.

Notes:
- This script uses standardized anomalies (z-scores) for SPI and temperature.
- VCI is computed as per-month historical min/max normalization (0-100).
- A simple weighted composite combines indicators.

Dependencies:
- earthengine-api (ee)
- geemap (optional, for interactive maps)
- folium (optional)

Example run: set target_year=2023, target_month=11 to reproduce an example
for November 2023.

In [1]:
# ----------------------- Short README / Guidance ---------------------------
# Usage notes / validation suggestions:
# - Validate thresholds with local in-situ observations or agricultural yield records.
# - Tune weights (WEIGHT_SPI, WEIGHT_VCI, WEIGHT_TEMP) to reflect local sensitivity.
# - Replace z-score SPI with full SPI gamma-distribution estimation for formal SPI.
# - Consider drought persistence by adding lagged months (e.g., 3-month SPI).
# - Automate by scheduling monthly runs and exporting to Drive or sending notifications.

In [2]:
import ee
import datetime

# Replace by your project numbber:
my_project = 'my-project-00000000000'

# Initialize Earth Engine. For first-time use, run `ee.Authenticate()` in a
# separate environment and then `ee.Initialize()`.
try:
    ee.Initialize(project=my_project)
except Exception as e:
    raise RuntimeError("Earth Engine not initialized. Run ee.Authenticate() first.")

In [3]:
# ----------------------------- Parameters ---------------------------------
# AOI: simplified rectangle covering a small agricultural region in Rondônia
study_area = ee.Geometry.Rectangle([-65.5, -13.5, -59.5, -7.5])

# Baseline period for climatology
baseline_start = '2001-01-01'
baseline_end = '2020-12-31'

# Target month to analyze (example: November 2023)
target_year = 2023
target_month = 11

# Weights for composite index (sum to 1)
WEIGHT_SPI = 0.5
WEIGHT_VCI = 0.35
WEIGHT_TEMP = 0.15

In [4]:
# ----------------------------- Helpers ------------------------------------
def month_start_end(year, month):
    start = datetime.date(year, month, 1)
    if month == 12:
        end = datetime.date(year + 1, 1, 1)
    else:
        end = datetime.date(year, month + 1, 1)
    return start.isoformat(), end.isoformat()

# Convert Python date strings to EE.Date

def ee_date(s):
    return ee.Date(s)

# Z-score normalization for an ee.Image: (x - mean)/std. Returns image with band 'z'
def zscore(image, mean_img, std_img, band):
    return image.select([band]).subtract(mean_img.select([band])).divide(std_img.select([band])).rename([band + '_z'])




In [5]:
# ------------------------- Data ingestion ---------------------------------
# CHIRPS daily precipitation: dataset id in GEE
# Note: CHIRPS daily collection ID is 'UCSB-CHG/CHIRPS/DAILY'
chirps_id = 'UCSB-CHG/CHIRPS/DAILY'
chirps = ee.ImageCollection(chirps_id)

# MODIS NDVI: use MOD13Q1 (16-day, 250m, NDVI band 'NDVI')
modis_id = 'MODIS/006/MOD13Q1'
modis = ee.ImageCollection(modis_id)

# ERA5-Land hourly temperature (2m) -- use hourly, will aggregate to daily/monthly
# Band: 'temperature_2m' (units: Kelvin)
era5_id = 'ECMWF/ERA5_LAND/HOURLY'
era5 = ee.ImageCollection(era5_id)

C:\Users\colla\AppData\Local\Programs\Python\Python313\Lib\site-packages\ee\deprecation.py:209: DeprecationWarning: 

Attention required for MODIS/006/MOD13Q1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1

  warnings.warn(warning, category=DeprecationWarning)


In [6]:
# ----------------------- Prepare target period -----------------------------
start_date, end_date = month_start_end(target_year, target_month)
print(f"Analyzing period: {start_date} to {end_date}")

Analyzing period: 2023-11-01 to 2023-12-01


In [7]:
# ----------------------- Precipitation: Monthly SPI (z-score) -------------
# 1) Sum daily precipitation for the target month
precip_month = chirps.filterDate(start_date, end_date).select('precipitation')
precip_month_sum = precip_month.sum().rename('precip')

In [8]:
# 2) Compute baseline climatology: monthly sums for same calendar month across baseline
# Build a list of monthly summed images for each baseline year

def monthly_sum_for_year(year):
    s, e = month_start_end(year, target_month)
    return chirps.filterDate(s, e).select('precipitation').sum().set('year', year)

baseline_years = list(range(int(baseline_start[:4]), int(baseline_end[:4]) + 1))
baseline_monthly = ee.ImageCollection([monthly_sum_for_year(y) for y in baseline_years])

precip_mean = baseline_monthly.mean().rename('precip_mean')
precip_std = baseline_monthly.reduce(ee.Reducer.stdDev()).rename('precip_std')

# Avoid division by zero
precip_std = precip_std.max(1e-6)

spi_z = zscore(precip_month_sum, precip_mean, precip_std, 'precip')  # positive means wetter than average
# For drought we want negative SPI -> invert sign so higher positive values = drought severity
spi_anomaly = spi_z.multiply(-1).rename('SPI_z')

In [9]:
# ----------------------- MODIS NDVI: VCI ---------------------------------
# MOD13Q1 NDVI is scaled by factor 0.0001.
# Select the MODIS image(s) overlapping the target month: use MODIS composites that overlap the period
modis_month = modis.filterDate(start_date, end_date).select('NDVI')
# Some months may contain 1 or 2 16-day composites; compute mean NDVI for the period
ndvi_month_mean = modis_month.mean().multiply(0.0001).rename('NDVI')

# Baseline for NDVI: compute per-month historical min and max across baseline years

def modis_month_for_year(year):
    s, e = month_start_end(year, target_month)
    return modis.filterDate(s, e).select('NDVI').mean().multiply(0.0001).set('year', year)

baseline_ndvi_collection = ee.ImageCollection([modis_month_for_year(y) for y in baseline_years])
ndvi_min = baseline_ndvi_collection.min().rename('NDVI_min')
ndvi_max = baseline_ndvi_collection.max().rename('NDVI_max')

# Compute VCI = (NDVI - NDVI_min) / (NDVI_max - NDVI_min) * 100
vci_denom = ndvi_max.subtract(ndvi_min).max(1e-6)
vci = ndvi_month_mean.subtract(ndvi_min).divide(vci_denom).multiply(100).rename('VCI')
# Clip to 0-100
vci = vci.clamp(0, 100)

# For composite, convert VCI to a drought-oriented anomaly where lower VCI -> higher drought
# We'll standardize VCI by converting to a negative z-score relative to baseline mean/std
ndvi_mean = baseline_ndvi_collection.mean().rename('NDVI_mean')
ndvi_std = baseline_ndvi_collection.reduce(ee.Reducer.stdDev()).rename('NDVI_std')
ndvi_std = ndvi_std.max(1e-6)
ndvi_z = zscore(ndvi_month_mean, ndvi_mean, ndvi_std, 'NDVI')
# Lower NDVI -> negative z -> multiply -1 to make positive values indicate drought
vci_anomaly = ndvi_z.multiply(-1).rename('VCI_z')

In [10]:
# ----------------------- ERA5 Temperature anomaly (z-score) ----------------
# ERA5 temperature is in Kelvin; convert to Celsius for interpretability
# Aggregate hourly -> daily mean -> monthly mean for target period
era5_month = era5.filterDate(start_date, end_date).select('temperature_2m')
# Mean over month (in Kelvin)
temp_month_mean_k = era5_month.mean().rename('temp_k')
temp_month_mean_c = temp_month_mean_k.subtract(273.15).rename('temp_c')

# Baseline: monthly means across baseline years

def era5_month_for_year(year):
    s, e = month_start_end(year, target_month)
    return era5.filterDate(s, e).select('temperature_2m').mean().rename('temp_k').set('year', year)

baseline_temp_collection = ee.ImageCollection([era5_month_for_year(y) for y in baseline_years])
baseline_temp_mean_k = baseline_temp_collection.mean().rename('temp_k_mean')
baseline_temp_std_k = baseline_temp_collection.reduce(ee.Reducer.stdDev()).rename('temp_k_std')

baseline_temp_mean_c = baseline_temp_mean_k.subtract(273.15).rename('temp_mean_c')
baseline_temp_std_c = baseline_temp_std_k.rename('temp_std_k')  # std in K similar magnitude
baseline_temp_std_c = baseline_temp_std_c.max(1e-6)

# Z-score for temp: higher positive means hotter than usual -> drought intensifier
temp_z = zscore(temp_month_mean_k, baseline_temp_mean_k, baseline_temp_std_k, 'temp_k').rename('temp_z')
# Keep as positive for drought
temp_anomaly = temp_z.rename('TEMP_z')

In [11]:
# ----------------------- Composite Drought Index ---------------------------
# Stack standardized layers (SPI_z, VCI_z, TEMP_z) into one image
composite = ee.Image.cat([spi_anomaly, vci_anomaly, temp_anomaly])

# Weighted sum
composite_weighted = composite.select('SPI_z').multiply(WEIGHT_SPI)
composite_weighted = composite_weighted.add(composite.select('VCI_z').multiply(WEIGHT_VCI))
composite_weighted = composite_weighted.add(composite.select('TEMP_z').multiply(WEIGHT_TEMP)).rename('CDI')

# ----------------------- Classification -----------------------------------
# Define thresholds on the composite index for drought severity.
# These thresholds are illustrative; adjust empirically based on local validation.
# CDI <= 0.5: No drought; 0.5-1.5 mild; 1.5-2.5 moderate; >2.5 severe

def classify_cdi(cdi):
    none = cdi.lte(0.5)
    mild = cdi.gt(0.5).And(cdi.lte(1.5))
    moderate = cdi.gt(1.5).And(cdi.lte(2.5))
    severe = cdi.gt(2.5)
    # create a single-band classified image: 0 none, 1 mild, 2 moderate, 3 severe
    return none.multiply(0).add(mild.multiply(1)).add(moderate.multiply(2)).add(severe.multiply(3)).rename('drought_class')

cdi_img = composite_weighted
drought_class = classify_cdi(cdi_img)

In [12]:
# ----------------------- Map & Export (example) ---------------------------
# For quick visualization in Python, use geemap or export to Asset/Drive. Below
# we create URLs for map visualization via folium (if geemap installed this is easier).

# Compute mean values over AOI for a quick report
stats = cdi_img.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=study_area,
    scale=5000,
    maxPixels=1e9
)

cdi_mean = stats.get('CDI')
print('Composite drought index (mean over AOI):', cdi_mean)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
# Example export to Google Drive (uncomment to use)
export_task = ee.batch.Export.image.toDrive(
     image=cdi_img.clip(study_area),
     description=f'CDI_{target_year}_{target_month}',
     folder='GEE_Exports',
     fileNamePrefix=f'CDI_{target_year}_{target_month}',
     region=study_area.bounds().getInfo()['coordinates'],
     scale=5000,
     maxPixels=1e13
)
export_task.start()

In [14]:




# ----------------------- Timeseries example --------------------------------
# Build a simple timeseries of monthly CDI for the last 24 months for AOI

def monthly_cdi_for(year, month):
    s, e = month_start_end(year, month)
    # repeat the same pipeline: precip sum, ndvi mean, temp mean, then composite
    prec_sum = chirps.filterDate(s, e).select('precipitation').sum()
    # baseline stats for SPI z: reuse baseline images computed previously but careful
    # For simplicity here compute z using baseline_monthly (already defined above)
    spi_z_local = zscore(prec_sum, precip_mean, precip_std, 'precip').multiply(-1)

    ndvi_m = modis.filterDate(s, e).select('NDVI').mean().multiply(0.0001)
    ndvi_z_local = zscore(ndvi_m, ndvi_mean, ndvi_std, 'NDVI').multiply(-1)

    temp_m_k = era5.filterDate(s, e).select('temperature_2m').mean()
    temp_z_local = zscore(temp_m_k, baseline_temp_mean_k, baseline_temp_std_k, 'temp_k')

    comp = spi_z_local.multiply(WEIGHT_SPI).add(ndvi_z_local.multiply(WEIGHT_VCI)).add(temp_z_local.multiply(WEIGHT_TEMP)).rename('CDI')
    mean_val = comp.reduceRegion(ee.Reducer.mean(), study_area, scale=5000).get('CDI')
    return ee.Feature(None, {'date': f"{year}-{month:02d}", 'CDI': mean_val})

# Create list for last 24 months
end_dt = datetime.date(target_year, target_month, 1)
months = []
for i in range(23, -1, -1):
    dt = end_dt - datetime.timedelta(days=30 * i)  # approximate stepping by 30 days
    months.append((dt.year, dt.month))

features = [monthly_cdi_for(y, m) for (y, m) in months]
feature_collection = ee.FeatureCollection(features)

# Get timeseries values (note: this executes server-side; may require waiting)



In [15]:
# Example: print the timeseries values (server call)
try:
    ts = feature_collection.getInfo()
    print('\nTimeseries (last 24 samples):')
    for f in ts['features']:
        print(f"{f['properties']['date']}: {f['properties']['CDI']}")
except Exception as e:
    print('Could not fetch timeseries synchronously in this environment. Use .getInfo() in your runtime or export results to Drive/Asset.')





Could not fetch timeseries synchronously in this environment. Use .getInfo() in your runtime or export results to Drive/Asset.


In [16]:
print('\nScript complete. Adjust parameters at the top (AOI, target_year, target_month) to run for a different period.')


Script complete. Adjust parameters at the top (AOI, target_year, target_month) to run for a different period.
